# If the user does not want to install NCMW and run ncmw_community, he can create a hierarchical community model using the codes defined in GitHub as follows:

In [1]:
import cobra

/Users/reihan/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
from libsbml import *
import libsbml

In [3]:
class HierarchicalModel():
    def __init__(self, name="Community"):
        sbmlns = SBMLNamespaces(3,1)
        sbmlns.addPackageNamespace("comp",1)
        sbmlns.addPackageNamespace("fbc",2)
        document = SBMLDocument(sbmlns)
        document.setPackageRequired("comp", True)
        document.setPackageRequired("fbc", False)

        self.sbml_doc = document 
        self.model = self.sbml_doc.createModel(name)
        self.model.setName(name)
        self.model.getPlugin("fbc").setStrict(True)
        self.models = []
        self.models_ref = []

    def add_model(self, id:str, source:str):
        m1 = ExternalModelDefinition(3,1)
        m1.setId(id)
        m1.setSource(source)
        dplugin = self.sbml_doc.getPlugin("comp")
        dplugin.addExternalModelDefinition(m1)

        smodel = Submodel()
        smodel.setModelRef(id)
        smodel.setId(id)
        mplugin = self.model.getPlugin("comp")
        mplugin.addSubmodel(smodel)

        reader = SBMLReader()
        doc = reader.readSBML(source)
        self.models.append(doc.getModel())
        self.models_ref.append(id)

    def getSubmodel(self, idx):
        return self.model.getPlugin("comp").getSubmodel(idx)

    def getModel(self,idx):
        return self.models[idx]

    def getReactionsFromModel(self, idx):
        return [r.getId() for r in self.models[idx].getListOfReactions()]

    def getMetabolitesFromModel(self, idx):
        return [e.getId() for e in self.models[idx].getListOfSpecies()]

    def getCompartmentsFromModel(self,idx):
        return [c.getId() for c in self.models[idx].getListOfCompartments()]

    def getExchangesFromModel(self, idx):
        rec = self.getReactionsFromModel(idx)
        i = 0
        while i < len(rec):
            id = rec[i]
            if not ("EX_" in id and "_e" in id):
                del rec[i]
            else:
                i += 1
        return rec

    def getExternalMetabolites(self,idx):
        met = self.getMetabolitesFromModel(idx)
        i = 0
        while i < len(met):
            id = met[i]
            if not (id[-2:] == "_e"):
                del met[i]
            else:
                i += 1
        return met

    def getAllExchanges(self):
        ex = []
        for idx in range(len(self.models)):
            ex.extend(self.getExchangesFromModel(idx))
        return list(set(ex))
    
    def getAllExternalMetabolites(self):
        ex = []
        for idx in range(len(self.models)):
            ex.extend(self.getExternalMetabolites(idx))
        return list(set(ex))

    def getAllBiomassFunctons(self):
        biomass_functions = []
        for ids, m in zip(self.models_ref, self.models):
            mfbc = m.getPlugin("fbc")
            o = mfbc.getObjective(0)
            reaction_id = o.getFluxObjective(0).getReaction()
            biomass_functions.append(ids + "__" + reaction_id)
        return biomass_functions

    def getModelRefContainMetabolites(self, met:str):
        refs = []
        for i in range(len(self.models)):
            if met in self.getMetabolitesFromModel(i):
                refs.append(self.models_ref[i])
        return refs

    def collapse_compartments(self, name:str, model_ref:str, compartments:list, **kwargs):
        c = self.add_compartment(name, **kwargs)
        for comp in compartments:
            self.replace(c, model_ref, comp)
        return c

    def removeAllExchangesFromModel(self, idx):
        s1 = test.getSubmodel(idx)
        exchanges = self.getExchangesFromModel(idx)
        for e in exchanges:
            self.delete(s1, e)

    def addExternalEnvironment(self, shuttle_lower_bound=-50, shuttle_upper_bound=50):
        _ = self.add_compartment("e", "external")
        mets = self.getAllExternalMetabolites()
        for m in mets:
            self.add_metabolite(m, "e")
            within_models = self.getModelRefContainMetabolites(m)
            _ = self.add_reaction("EX_" + m, {m:1}, dict())
            for model in within_models:
                _ = test.add_reaction(f"SH__{model}_"+m, {m:1}, {f"{model}__{m}":1}, lower_bound=shuttle_lower_bound, upper_bound=shuttle_upper_bound)


    def delete(self,submodel, id_ref:str):
        deleted = Deletion()
        deleted.setIdRef(id_ref)
        submodel.addDeletion(deleted)

    def replace(self, element, model_ref:str, id_ref:str):
        replaced = ReplacedElement()
        replaced.setIdRef(id_ref)
        replaced.setSubmodelRef(model_ref)
        element.getPlugin("comp").addReplacedElement(replaced)

    def setCommunityObjective(self, coefficients, biomass_reactions):
        mfbc = self.model.getPlugin("fbc")
        objective = mfbc.createObjective()
        objective.setId("Growth")
        objective.setType("maximize")
        mfbc = self.model.getPlugin("fbc")
        mfbc.setActiveObjectiveId("Growth")

        for c, b in zip(coefficients, biomass_reactions):
            fluxObjective = objective.createFluxObjective()
            fluxObjective.setReaction(b)
            fluxObjective.setCoefficient(c)

    #def add_constraint(self, id, )

    def add_compartment(self,id,name="", meta_id="", constant=True, size=1):
        comp = self.model.createCompartment()
        comp.setId(id)
        comp.setName(name)
        comp.setMetaId(meta_id)
        comp.setConstant(constant)
        comp.setSize(size)
        return comp

    def add_parameter(self, id:str, value, constant=True):
        p = Parameter(3,1)
        p.setId(id)
        p.setValue(value)
        p.setConstant(constant)
        self.model.addParameter(p)

    def add_metabolite(self, id:str, compartment:str, boundary:bool=False, substance_units:bool=True, constant:bool=True):
        species = self.model.createSpecies()
        species.setId(id)
        species.setCompartment(compartment)
        species.setHasOnlySubstanceUnits(substance_units)
        species.setBoundaryCondition(boundary)
        species.setConstant(constant)
        return species

    def add_reaction(self, id:str, reactants:dict, products:dict, lower_bound=-10, upper_bound=1000,reversible=True, constant=False):
        if not isinstance(reactants, dict):
            raise ValueError()
        if not isinstance(products, dict):
            raise ValueError()
        reaction = self.model.createReaction()
        reaction.setId(id)
        reaction.setReversible(reversible)
        for key,val in reactants.items():
            reactant = reaction.createReactant()
            reactant.setSpecies(key)
            reactant.setStoichiometry(val)
            reactant.setConstant(constant)
        for key,val in products.items():
            product = reaction.createProduct()
            product.setSpecies(key)
            product.setStoichiometry(val)
            product.setConstant(constant)

        self.add_parameter(id+ "_lb", lower_bound)
        self.add_parameter(id+ "_ub", upper_bound)

        rplug = reaction.getPlugin("fbc")
        rplug.setLowerFluxBound(id+"_lb")
        rplug.setUpperFluxBound(id + "_ub")
        
        return reaction

    def convert_to_cobra(self, path="test.xml"):
        self.save_flatten(path)
        model = cobra.io.read_sbml_model(path)
        return model

    def reset(self):
        self.__init__(self.model.getName())

    def save_flatten(self, path):
        sbmldoc = self.sbml_doc

        props = ConversionProperties()
        props.addOption('flatten comp', True) 
        converter = libsbml.CompFlatteningConverter()
        converter.setDocument(sbmldoc)

        status = converter.convert()
        if status != 0:
            raise ValueError("Not a valid hierarchical model")

        writer  = SBMLWriter()
        writer.writeSBML(sbmldoc, path)        

    def save(self, path):
        writeSBMLToFile(self.sbml_doc, path)

In [4]:
sbmlns = SBMLNamespaces(3,1,"comp",1)
sbmlns.addPackageNamespace("fbc",2)
document = SBMLDocument(sbmlns)
document.setPackageRequired("comp", True)
document.setPackageRequired("fbc", False)

model = document.createModel("community_model")
mfbc = model.getPlugin("fbc")
mfbc.setStrict(True)

# Compartment
comp = model.createCompartment()
comp.setSize(1)
comp.setConstant(True)
comp.setId("exchanges")

# Plugins
cplugin = comp.getPlugin("comp")
dplugin = document.getPlugin("comp")
mplugin = model.getPlugin("comp")

# Models
model_paths = ["iFap484_fbc2.xml", "iBif452_fbc2.xml"]
external_species = []
present_in_models = []
exchanges = []
ids = []
for path in model_paths:

    id = path.split(".")[0]
    ids.append(id)
    m1 = ExternalModelDefinition(3,1)
    m1.setId(id)
    m1.setSource(path)
    dplugin.addExternalModelDefinition(m1)

    r = SBMLReader()
    m = r.readSBML(path).getModel()

    ex = []
    for s in m.getListOfReactions():
        a = s.getPlugin("fbc")
        i = s.getId()
        if "_EX_" in i:
            ex.append(i)
            met_id = "M_" + i[5:]
            if met_id not in external_species:
                external_species.append(met_id)
                present_in_models.append([id])
            else:
                present_in_models[external_species.index(met_id)] += [id]

    exchanges.append(ex)
    smodel = Submodel()
    smodel.setModelRef(id)
    smodel.setId(id)
    mplugin.addSubmodel(smodel)


# Collect all exchanges in single compartment (still seperated)
# replaced = ReplacedElement()
# replaced.setIdRef("C_e")
# replaced.setSubmodelRef(ids[0])
# cplugin.addReplacedElement(replaced)

# replaced = ReplacedElement()
# replaced.setIdRef("e")
# replaced.setSubmodelRef(ids[1])
# cplugin.addReplacedElement(replaced)

# Create environment
# external = model.createCompartment()
# external.setSize(1)
# external.setConstant(True)
# external.setId("environment")
# species = model.createSpecies()
# species.setId("met_e")
# species.setCompartment("environment")
# species.setHasOnlySubstanceUnits(True)
# species.setBoundaryCondition(True)
# species.setConstant(False)

# Shuttle reactions

# shuttle = model.createReaction()
# shuttle.setId("SH_h2o_e")
# s_pkg = shuttle.getPlugin("comp")
# shuttle.setReversible(True)
# reactant = shuttle.createReactant()
# reactant.setSpecies()

for id,ex in zip(ids,exchanges):
    for e in ex:
        pass

#External metabolites
for met_e, ms in zip(external_species, present_in_models):
    species = model.createSpecies()
    species.setId(met_e)
    species.setCompartment("exchanges")
    species.setHasOnlySubstanceUnits(True)
    species.setBoundaryCondition(True)
    species.setConstant(False)
    splugin = species.getPlugin("comp")
    for idx in ids:
        if idx in ms:
            s1 = ReplacedElement()
            s1.setIdRef(met_e[2:])
            s1.setSubmodelRef(idx)
            splugin.addReplacedElement(s1)
            #t1 = Deletion()
            #t2 = Deletion()
            #t3 = Deletion()
            #t1.setIdRef("R_EX_m2")
            #t2.setIdRef("iFap484_fbc2__R_EX_m2")
            #t3.setIdRef("iBif452_fbc2__R_EX_m2")
            #smodel.addDeletion(t1)
            #smodel.addDeletion(t2)
            #smodel.addDeletion(t3)
        


objective = mfbc.createObjective()
objective.setId("Growth")
objective.setType("maximize")

mfbc.setActiveObjectiveId("Growth")

fluxObjective = objective.createFluxObjective()
fluxObjective.setReaction("iFap484_fbc2__R_Biomass")
fluxObjective.setCoefficient(1)

fluxObjective = objective.createFluxObjective()
fluxObjective.setReaction("iBif452_fbc2__R_Biomass")
fluxObjective.setCoefficient(1)




writeSBMLToFile(document, "test.xml")

1

In [5]:
reader  = SBMLReader()
sbmldoc = reader.readSBML("test.xml")
sbmldoc.getErrorLog().toString()

''

In [6]:
props = ConversionProperties()
props.addOption('flatten comp', True) 

In [7]:
converter = libsbml.CompFlatteningConverter()
converter.setDocument(sbmldoc)

0

In [8]:
converter.convert()

0

In [9]:
writer  = SBMLWriter()
writer.writeSBML(sbmldoc, "flattened_model_test.xml")

True

# The community model is created and converted to be readable in cobra. 
When we have this model as a community, we can use all features applicable in Cobrapy, such as changing the medium, knockout genes, reactions, etc.

In [10]:
import cobra
model = cobra.io.sbml.read_sbml_model("flattened_model_test.xml")
model

Adding exchange reaction EX_m2 with default bounds for boundary metabolite: m2.
Adding exchange reaction EX_m4 with default bounds for boundary metabolite: m4.
Adding exchange reaction EX_m11 with default bounds for boundary metabolite: m11.
Adding exchange reaction EX_m14 with default bounds for boundary metabolite: m14.
Adding exchange reaction EX_m16 with default bounds for boundary metabolite: m16.
Adding exchange reaction EX_m18 with default bounds for boundary metabolite: m18.
Adding exchange reaction EX_m21 with default bounds for boundary metabolite: m21.
Adding exchange reaction EX_m24 with default bounds for boundary metabolite: m24.
Adding exchange reaction EX_m27 with default bounds for boundary metabolite: m27.
Adding exchange reaction EX_m33 with default bounds for boundary metabolite: m33.
Adding exchange reaction EX_m35 with default bounds for boundary metabolite: m35.
Adding exchange reaction EX_m39 with default bounds for boundary metabolite: m39.
Adding exchange reac

Adding exchange reaction EX_m773 with default bounds for boundary metabolite: m773.
Adding exchange reaction EX_m774 with default bounds for boundary metabolite: m774.
Adding exchange reaction EX_m777 with default bounds for boundary metabolite: m777.
Adding exchange reaction EX_m779 with default bounds for boundary metabolite: m779.


Name,community_model
Memory address,0x07fd99acd4f28
Number of metabolites,1647
Number of reactions,1642
Number of groups,221
Objective expression,1.0*iBif452_fbc2__R_Biomass - 1.0*iBif452_fbc2__R_Biomass_reverse_ce6f7 +...
Compartments,"exchanges, Extraorganism, b, Cytosol, Extraorganism, b, Cytosol"


In [12]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
iBif452_fbc2__M_m14,iBif452_fbc2__R_EX_m14,0.01489,0,0.00%
iBif452_fbc2__M_m16,iBif452_fbc2__R_EX_m16,0.1684,0,0.00%
iBif452_fbc2__M_m2,iBif452_fbc2__R_EX_m2,1,0,0.00%
iBif452_fbc2__M_m372,iBif452_fbc2__R_EX_m372,8.835E-05,0,0.00%
iBif452_fbc2__M_m41,iBif452_fbc2__R_EX_m41,0.003552,0,0.00%
iBif452_fbc2__M_m44,iBif452_fbc2__R_EX_m44,0.005961,0,0.00%
iBif452_fbc2__M_m47,iBif452_fbc2__R_EX_m47,0.01641,0,0.00%
iBif452_fbc2__M_m50,iBif452_fbc2__R_EX_m50,0.01127,0,0.00%
iBif452_fbc2__M_m53,iBif452_fbc2__R_EX_m53,0.01567,0,0.00%
iBif452_fbc2__M_m59,iBif452_fbc2__R_EX_m59,0.01147,0,0.00%


# Suppose the user installed NCMW and is sure about requirement solvers such as cplex or glpk. In that case, he can only import the hierarchical package and run a few line codes, as was explained in the documentation attached to the GitHub repository. 

In [ ]:
from ncmw.community import HierarchicalCommunityModel

In [ ]:
model = HierarchicalCommunityModel(["../data/models/model1.xml", "../data/models/model2.xml"])